# Embedding Berufe to 300d Glove vocab, then clusetring
### https://deepset.ai/german-word-embeddings

In [5]:
from pathlib import Path
import pandas as pd
import numpy as np
import os

In [6]:
# Pfad zum Hauptverzeichnis des Projekts
PROJECT_ROOT = Path(os.getcwd()).parent

DATA_PATH = PROJECT_ROOT / 'data'
PLOT_PATH = PROJECT_ROOT / 'plots'

CURRENT_YEAR = '2024'  # oder '2021' für die alten Daten

DF_MDB_PATH = DATA_PATH / CURRENT_YEAR / 'output' / 'df_mdb.csv'

In [8]:
df_berufe = pd.read_csv(DF_MDB_PATH)

In [9]:
df_berufe.head()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,16,1,17,18,19,20,GEBURSDATUM_DATE,RELIGION_MAPPED,FAMILIENSTAND_MAPPED,BERUF_MAPPED
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,UNBEKANNT,17.01.2008,männlich,keine Angaben,katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,katholisch,keine Angaben,Jurist*in
1,11000002,Achenbach,Ernst,09.04.1909,Siegen,UNBEKANNT,02.12.1991,männlich,"verheiratet, 3 Kinder",evangelisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evangelisch,"verh., 3 Kinder",Jurist*in
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,katholisch,"verh., >3 Kinder",hilfsreferent
3,11000004,Ackermann,Else,06.11.1933,Berlin,UNBEKANNT,14.09.2019,weiblich,ledig,evangelisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evangelisch,ledig,Arzt/Ärztin
4,11000005,Adam,Ulrich,09.06.1950,Teterow,UNBEKANNT,UNBEKANNT,männlich,"verheiratet, 2 Kinder",evangelisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,evangelisch,"verh., 2 Kinder",Naturwissenschaftler*in


In [10]:
df_berufe[df_berufe['NACHNAME'] == 'Maas'][['VORNAME', 'NACHNAME', 'BERUF', 'BERUF_MAPPED']]

,VORNAME,NACHNAME,BERUF,BERUF_MAPPED
3956,Heiko,Maas,Jurist,Jurist*in


In [11]:
df_berufe[['ID','BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False).head()

,ID
BERUF,
Rechtsanwalt,202
Geschäftsführer,82
Landwirt,80
Angestellter,64
Rechtsanwalt und Notar,47


In [12]:
len(set(df_berufe.BERUF))

2319

In [18]:
import sys
sys.path.append("..") # TODO: this is not pretty!
from importlib import reload
import src.berufe_mapping as berufe_mapping
reload(berufe_mapping)

<module 'src.berufe_mapping' from '/Users/lena/code/bundestag-mining/notebooks/../src/berufe_mapping.py'>

In [22]:
# already reduction: 2.2k => 0.9
len(set(df_berufe.BERUF_MAPPED))

1206

In [21]:
df_berufe.BERUF_MAPPED = df_berufe.BERUF.copy()
df_berufe = berufe_mapping.basic_cleaning_berufe(df_berufe, 'BERUF_MAPPED')
df_berufe[['ID','BERUF_MAPPED']].groupby('BERUF_MAPPED').count().sort_values(by='ID', ascending=False).head()

,ID
BERUF_MAPPED,
rechtsanwalt,397
geschäftsführer,109
landwirt,107
-ingenieur,99
jurist,90


In [23]:
NUM_FREQ_JOBS = 256
frequent_jobs = df_berufe[['ID','BERUF_MAPPED']].groupby('BERUF_MAPPED').count().sort_values(by='ID', ascending=False).head(NUM_FREQ_JOBS).index
frequent_jobs_list = [job for job in frequent_jobs.str.lower()]
list(frequent_jobs_list)[:8]

['rechtsanwalt',
 'geschäftsführer',
 'landwirt',
 '-ingenieur',
 'jurist',
 'lehrer',
 '-volkswirt',
 'angestellter']

In [24]:
# embedding source https://deepset.ai/german-word-embeddings
glove_path = os.path.join('data', 'GloVe_vectors_de.txt')

In [25]:
embeddings_index = {}
with open(glove_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

FileNotFoundError: [Errno 2] No such file or directory: 'data/GloVe_vectors_de.txt'

In [ ]:
coefs.shape

In [ ]:
embeddings_index['baum'].shape

In [ ]:
job_dict = {}
for job in frequent_jobs_list:
    if job in embeddings_index.keys():
        job_dict[job] = embeddings_index[job]
    else:
        print(f'sorry not found: {job}')

In [ ]:
len(job_dict)

In [ ]:
df_job_dict = pd.DataFrame(job_dict).T
df_job_dict

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

NCLUSTER = 32
kmeans = KMeans(n_clusters=NCLUSTER).fit(df_job_dict)
#clustering = DBSCAN(eps=3, min_samples=2).fit(df_job_dict)


In [ ]:
labels = kmeans.labels_
len(labels)

In [ ]:
for cluster in range(NCLUSTER):
    print()
    print(f'cluster number {cluster}')
    print(df_job_dict.iloc[labels==cluster].index)

In [ ]:
df_job_dict.iloc[labels==6].index